In [ ]:
import pandas as pd
from time import sleep
from datetime import datetime as dt
import re
import libDatabase as db                  #Script used as Library, build on PyMySql to insert data in Mysql Tables and handle exceptions. 
from tabulate import tabulate
import matplotlib.pyplot as plt
import seaborn as sns
from gazpacho import get, Soup

%matplotlib inline

#### Credentials for DB

In [ ]:

host="localhost"
port=3306
user_name = "bhavesh1"
password = "Pass#123"
schema_="L00150830"

#db.open_database(host=host, port=port, user_name = user_name,password = password, schema_=schema_)


In [ ]:
URL = "https://en.wikipedia.org/wiki/List_of_prime_ministers_of_the_United_Kingdom#18th_century"
URL_1 = "https://en.wikipedia.org"

soup = Soup(get(URL))
data = soup.find("table")
imp_data = data[1].find("tr")[2:]

In [ ]:
rank = "Earl|Duke|Marquess|Lord|Viscount|Baron|Sir"

prime_minister = []
prime_minister_2 = []
prime_minister_1 = []
year_of_start = []
year_of_end = []
party = []
URL_S = []



#### Scraping PM, Start Date, End Date, Rank and links for wikipedia page of individual PM

In [ ]:

i = 0
list_i = []
while i < len(imp_data)-1:
    #print (i)
    rowspan = imp_data[i].find("td")[2].attrs.get("rowspan")
    list_i.append(i)
    rowspan = int(rowspan)#if rowspan is not None else 1
    if type(imp_data[i].find("td")[2].find("a"))  is list:
        
        if imp_data[i].find("td")[2].find("a")[0].text == "Sir":
            #print("1sir")
            prime_minister = prime_minister + [imp_data[i].find("td")[2].find("a")[0].attrs.get("title")+", " +imp_data[i].find("td")[2].find("a")[0].text]
            
        else:
            prime_minister = prime_minister + [imp_data[i].find("td")[2].find("a")[0].attrs.get("title")]#* rowspan
        URL_S.append(URL_1 + imp_data[i].find("td")[2].find("a")[0].attrs.get("href"))
        #party = party + [imp_data[i].find("td")[6].text]
    
    else:
        
        URL_S.append(URL_1 + imp_data[i].find("td")[2].find("a").attrs.get("href"))
        if imp_data[i].find("td")[2].find("a").text == "Sir":
            #print("2sir", i)
            prime_minister = prime_minister + [imp_data[i].find("td")[2].find("a").attrs.get("title") +", " + imp_data[i].find("td")[2].find("a").text]
        else:    
            prime_minister = prime_minister + [imp_data[i].find("td")[2].find("a").attrs.get("title")]#* rowspan
        
        #party = party + [imp_data[i].find("td")[6].find("a").attrs.get("title")]#* rowspan
        #try:
            #party = party + [imp_data[i].find("td")[6].find("a").attrs.get("title")]#* rowspan
        #except:
            #party = party + [imp_data[i].find("td")[6].find("a")[0].attrs.get("title")]
            

    party.append(imp_data[i].find("td")[0].attrs.get("bgcolor"))
        
        
    if imp_data[i].find("td")[3].strip("br")[:-1] == "See also § Main articles:":
        
        year_of_start = year_of_start +[imp_data[i+1].find("td")[0].strip("br")] #* rowspan
        year_of_end = year_of_end+[imp_data[i+1].find("td")[1].strip("br")]
    else:
        year_of_start = year_of_start +[imp_data[i].find("td")[3].strip("br")] #* rowspan
        year_of_end = year_of_end+[imp_data[i].find("td")[4].strip("br")]
        
    i += rowspan
    


In [ ]:
prime_minister_1 = [name.split(",")[0] for name in prime_minister]

for name in prime_minister:
    if len(name.split(",")) > 1:
        prime_minister_2.append(re.findall(rank, name.split(",")[1])[0])
    else:
        prime_minister_2.append("No Rank")

year_of_start = [dt.strptime(date_time, '%d %B%Y').strftime("%Y-%m-%d") for date_time in year_of_start]
year_of_end[-1] = dt.now().strftime("%d %B%Y")
year_of_end = [dt.strptime(date_time, '%d %B%Y').strftime("%Y-%m-%d") for date_time in year_of_end ]

In [ ]:
new_labels = ['Whig', 'Tory', 'Conservative', 'Liberal', 'Labour']
old_labels = ["#FF7F00", "#3333CC", "#0087DC", "#ffd700", "#E4003B"]

df = pd.DataFrame({"Prime_Minister":prime_minister_1, "URL": URL_S,"Rank":prime_minister_2 ,"Party":party,"Year_of_start": year_of_start, "Year_of_end":year_of_end})
df['Party'] = df['Party'].replace(old_labels, new_labels)

df.head()


#### Looping through Wikipedia pages of every PM and scraping Date Of Brith

In [ ]:
DOB = []
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]


for idx, URL in enumerate (df.iloc[:,1]):
#URL = df.iloc[1,1]

    soup = Soup(get(URL))
    data = soup.find("table")
    #print(df.iloc[idx,0],"--" , idx)

    for dat in data:
        if dat.attrs.get("class") == "infobox vcard":
            if type(dat.find("tr")) is list:
                for row in dat.find("tr"):
                    if row.find("th") is not None:
                        if type(row.find("th")) is list:
                            for new_rows in row.find("th"):
                                if new_rows.find("th").text == "Born":
                                    #print(df.iloc[idx,0], "-->",row[0].find("td").strip("br"))
                                    DOB.append(df.iloc[idx,0], "-->",row[0].find("td").strip("br"))

                        else:
                            
                            if row.find("th").text == "Born":
                                D_O_B =  row.find("td").strip("br")
                                if D_O_B[D_O_B.find("(") + 1: D_O_B.find("(") + 11][0].isalpha() == False:
                                    #print(D_O_B[D_O_B.find("(") + 1: D_O_B.find("(") + 11])
                                    DOB.append(D_O_B[D_O_B.find("(") + 1: D_O_B.find("(") + 11])
                                
                                else:
                                    if D_O_B[D_O_B.find("(") + 1: D_O_B.find("(") + 11] == "c. 1674Com":
                                        #print("1674-01-01")
                                        DOB.append("1674-01-01")
                                    for month in months:
                                        if D_O_B.find(month) > 0:
                                            #print(dt.strptime(D_O_B[D_O_B.find(month) + -2: D_O_B.find(month) +len(month) +5], '%d %B %Y').strftime('%Y-%m-%d'))
                                            DOB.append(dt.strptime(D_O_B[D_O_B.find(month) + -2: D_O_B.find(month) +len(month) +5], '%d %B %Y').strftime('%Y-%m-%d'))
                                     
            else:
                
                print(df.iloc[idx,1])
                #print (data)
                print("-----------------")
                DOB.append("NA")



        
        


In [ ]:
df["Date_Of_Birth"] = DOB
#df["Date_Of_Death"] = DOD
#df_PM.to_csv("sample2.csv")
df.info()


In [ ]:
i = 0 
dates = []
Run_Flag = True

while Run_Flag:
    #print(i)
    if type(imp_data[i].find("td")) is list:
        length = len(imp_data[i].find("td"))
        if length > 5:
            rowspan = imp_data[i].find("td")[-2].attrs.get("rowspan")
            rowspan = int(rowspan) if rowspan is not None else 1
            dates.append(imp_data[i].find("td")[-2].strip("p"))
        else:
            rowspan = imp_data[i].find("td")[-1].attrs.get("rowspan")
            rowspan = int(rowspan) if rowspan is not None else 1
            dates.append(imp_data[i].find("td")[-1].strip("p"))
            
    else:
        if imp_data[i].find("td") is not None:
            rowspan = imp_data[i].find("td").attrs.get("rowspan")
            rowspan = int(rowspan) if rowspan is not None else 1
            dates.append(imp_data[i].find("td").strip("p"))
        else:
            dates.append(imp_data[i].find("th")[-1].strip("p"))

            
        
        
    #dates.append(imp_data[i].find("td")[-2].strip("p").replace(imp_data[0].find("td")[-2].text, "").replace(" ", "").split("–"))
    #dates.append(imp_data[i].find("td")[-2].strip("p"))
    i += rowspan
    if dates[-1] == "Elizabeth II 1952–present":
        dates[-1] = dates[-1].replace("present", "2020")
        Run_Flag = False

#dates_1 = [dates.split(" ")[:2] for dates in dates]
dates[-3] += "–1936"
dates

In [ ]:
monarch = list(map(lambda date: ''.join([i for i in date if i.isalpha()]), dates))
date_1 = list(map(lambda date: ''.join([i for i in date if i.isdigit()]), dates))
from_date = [date[:4] + "-01-01" for date in date_1]
to_date = [date[4:]+"-01-01" for date in date_1]

In [ ]:
df_monarch = pd.DataFrame({"Monarch": monarch, "Start_Date": from_date, "End_Date":to_date})
#df_monarch.End_Date = pd.to_datetime(df_monarch.End_Date)
#df_monarch.Start_Date = pd.to_datetime(df_monarch.Start_Date)
df_monarch.to_csv("monarch.csv")

df_monarch.head()

#### Inserting Data into DB

In [ ]:
from time import sleep
db.open_database(host=host, port=port, user_name = user_name,password = password, schema_=schema_)
sleep(2)
for indx, rows in df.iterrows():
    db.insert_person(rows["Prime_Minister"], rows["Rank"], rows["Party"], rows["Year_of_start"], rows["Year_of_end"], rows["Date_Of_Birth"])

for indx, rows in df_monarch.iterrows():  
    db.insert_monarch(rows['Monarch'],rows['Start_Date'], rows['End_Date'] )
db.close_database()

### Q1. Which political party has produced the most individual prime ministers?

In [ ]:
db.open_database(host=host, port=port, user_name = user_name,password = password, schema_=schema_)
out = db.db_cmd_execute("select distinct Party from C00265741.Persons ;")
list_of_parties = [party[0] for party in out]
count_of_parties = []
for party in list_of_parties:
    count_of_parties.append(db.db_cmd_execute("select count(*) from C00265741.Persons where Party = '"+ str(party) +"' ;")[0][0])

print("Political party has produced the most individual prime ministers is: \033[1m", list_of_parties[count_of_parties.index(max(count_of_parties))], "\033[0m")
db.close_database()

### Q2. Provide a list of prime ministers who served under each monarch.

In [ ]:
db.open_database(host=host, port=port, user_name = user_name,password = password, schema_=schema_)
list_of_monarch = db.db_cmd_execute("select Monarch from C00265741.Monarchs ;")
list_of_monarch = [monarch[0] for monarch in list_of_monarch]
prime_ministers_monarch = {}
prime_ministers_monarch_1 = {}

for monarch in list_of_monarch:
    prime_ministers = db.db_cmd_execute("SELECT * FROM C00265741.Persons \
            where StartDate > (SELECT StartDate FROM C00265741.Monarchs where Monarch = '"+str(monarch)+"') and \
            StartDate < (SELECT EndDate FROM C00265741.Monarchs where Monarch = '"+str(monarch)+"')")
    prime_ministers_monarch[monarch] = [minister[0] for minister in prime_ministers]
    
    prime_ministers = db.db_cmd_execute("SELECT * FROM C00265741.Persons \
            where EndDate > (SELECT StartDate FROM C00265741.Monarchs where Monarch = '"+str(monarch)+"') and \
            EndDate < (SELECT EndDate FROM C00265741.Monarchs where Monarch = '"+str(monarch)+"')")
    prime_ministers_monarch_1[monarch] = [minister[0] for minister in prime_ministers]

    
for key in prime_ministers_monarch:
    temp = []
    print("\033[1m" +key+ "\033[0m", end = ' -->  \n')

    for name in prime_ministers_monarch[key]:
        temp.append(name)
        
    for name in prime_ministers_monarch_1[key]:
        temp.append(name)
        
    for name in set (temp):
        pass
        print(name)
    print("--"*50)
    
db.close_database()

### Q3. Which political party held the office of prime minister for the longest amount of overall time, and for how long (in years, months, and days)?

#### Function for converting Days to Year, Month and Days

In [ ]:
def find_year( number_of_days ):
    if number_of_days is None:
        return(0, 0, 0)
    else:
        number_of_days = int(number_of_days)
        year = int(number_of_days / 365.25)
        months = int(round((number_of_days  % 365.25) / 30.436875, 0))
        days = int(round ((number_of_days  % 365.25) % 30.436875, 0))
        return(year, months, days)

In [ ]:
db.open_database(host=host, port=port, user_name = user_name,password = password, schema_=schema_)

number_of_days = []
for party in list_of_parties:
    term_start_end_dates = db.db_cmd_execute ("SELECT sum(datediff(EndDate, StartDate)) FROM C00265741.Persons where Party = '"+str(party)+"'")[0][0]
    number_of_days.append(term_start_end_dates)
duration = find_year(max(number_of_days))

print("Political party held the office of prime minister for the longest amount of overall time is: \033[1m", 
      list_of_parties[number_of_days.index(max(number_of_days))], "\033[0m \ni.e for ",duration[0]," Years ",duration[1]," Months and ",duration[2]," Days")

db.close_database()

### Q4. Which individual politician held the office of prime minister for the longest amount of uninterrupted time, and for how long (in years, months, and days)?

In [ ]:
db.open_database(host=host, port=port, user_name = user_name,password = password, schema_=schema_)

out = db.db_cmd_execute ("select PrimeMinister, datediff(EndDate, StartDate) from C00265741.Persons \
                    order by datediff(EndDate, StartDate) desc limit 1;")[0]

duration = find_year(out[1])

print("Prime Minister held the office of prime minister for the longest amount of of uninterrupted time is: \033[1m",out[0], "\033[0m \ni.e for ",duration[0]," Years ",duration[1]," Months and ",duration[2]," Days")

db.close_database()

### Q5. Which individual politician has held the office of prime ministers for the longest amount of overall time?

In [ ]:
db.open_database(host=host, port=port, user_name = user_name,password = password, schema_=schema_)

out = db.db_cmd_execute("select distinct PrimeMinister from C00265741.Persons ;")
list_of_PrimeMinister = [minister[0] for minister in out]
list_of_overall_time = []
for minister in list_of_PrimeMinister:
    out = db.db_cmd_execute("SELECT  sum(datediff(EndDate, StartDate))  FROM C00265741.Persons \
                                where PrimeMinister = '"+ str(minister)+"';")[0][0]
    list_of_overall_time.append(out)
    
df_temp = pd.DataFrame({"PM":list_of_PrimeMinister,"time":list_of_overall_time})
overall_longest = df_temp[df_temp.time == df_temp.time.max()].values[0]  
duration = find_year(int(overall_longest[1]))

print("Politician has held the office of prime ministers for the longest amount of overall time: \033[1m",overall_longest[0], "\033[0m \ni.e for ",duration[0]," Years ",duration[1]," Months and ",duration[2]," Days")

db.close_database()



### Q6. Which individual politician held the office of prime minister for the shortest amount of overall time, and for how long (in years, months, and days)?

In [ ]:
db.open_database(host=host, port=port, user_name = user_name,password = password, schema_=schema_)

out = db.db_cmd_execute("select distinct PrimeMinister from C00265741.Persons ;")
list_of_PrimeMinister = [minister[0] for minister in out]
list_of_overall_time = []
for minister in list_of_PrimeMinister:
    out = db.db_cmd_execute("SELECT  sum(datediff(EndDate, StartDate))  FROM C00265741.Persons \
                                where PrimeMinister = '"+ str(minister)+"';")[0][0]
    list_of_overall_time.append(out)
    
df_temp = pd.DataFrame({"PM":list_of_PrimeMinister,"time":list_of_overall_time})
overall_shortest = df_temp[df_temp.time == df_temp.time.min()].values[0]  
duration = find_year(int(overall_shortest[1]))

print("Politician has held the office of prime ministers for the shortest amount of overall time: \033[1m",overall_shortest[0], "\033[0m \ni.e for ",duration[0]," Years ",duration[1]," Months and ",duration[2]," Days")

db.close_database()


### Q7. List the prime ministers who where either an earl, viscount, duke, or knight?

In [ ]:
db.open_database(host=host, port=port, user_name = user_name,password = password, schema_=schema_)

ministers = db.db_cmd_execute("select PrimeMinister,Rank from C00265741.Persons \
                                where Rank = 'earl' or Rank = 'viscount' or Rank = 'duke' or \
                                Rank = 'Sir' order by Rank;")
print(tabulate(set(ministers),headers= ["Prime Minister", "Rank"], tablefmt="psql"))
    
db.close_database()

### Q8. What age was each prime minister on the day they assumed office?

In [ ]:
db.open_database(host=host, port=port, user_name = user_name,password = password, schema_=schema_)

out = db.db_cmd_execute("select PrimeMinister, StartDate, datediff(StartDate, DOB) from C00265741.Persons;")
out = [list(row[0:2])+ list(find_year(row[2])) for row in out]

print(tabulate(out, tablefmt="fancy_grid", headers=["Prime Minister", "Date of Assuming", "Age(Years)", "Age(Month)", "Age(Days)"]))  
db.close_database()

### Q9. On the last day of their term of office, which politician was the oldest?

In [ ]:
db.open_database(host=host, port=port, user_name = user_name,password = password, schema_=schema_)

out = db.db_cmd_execute("select PrimeMinister, EndDate, datediff(EndDate, DOB) from C00265741.Persons;")

    
df_temp = pd.DataFrame(out)
age = df_temp.loc[df_temp[2] == df_temp[2].max(), :].values[0]
duration = find_year(int(age[2]))

print("Oldest Politician on the last day of the term: \033[1m",age[0], "\033[0m \ni.e for ",duration[0]," Years ",duration[1]," Months and ",duration[2]," Days")

db.close_database()

### Q10. In your view, and based on statistics calculated from your scraped data, which UK political party is the most successful? Show and describe your reasoning.

In [ ]:
db.open_database(host=host, port=port, user_name = user_name,password = password, schema_=schema_)
total_number_of_ministers = []
for party in list_of_parties:
    out = db.db_cmd_execute("SELECT count(PrimeMinister) FROM C00265741.Persons where Party = '"+ str (party)+ "';")
    total_number_of_ministers.append(out[0][0])
db.close_database()

sns.set(font_scale=1.8)  # crazy big
f, axs = plt.subplots(1,2,figsize=(30,10))

axs[0].pie(x=number_of_days, autopct="%.2f%%",  explode=[0.05, 0.05, 0.15, 0.05, 0.05], labels=list_of_parties, pctdistance=0.7)
axs[0].set_title('Overall Ruling ',fontsize=20)


sns.barplot(x = total_number_of_ministers, y = list_of_parties, ax = axs[1])
axs[1].set_title('Number of PMs from each party',fontsize=20);

#### Conservative  has ruled UK for the longest amount of time. Moreover, Conservative had heighest number of PMs throughout the political history of UK. Thus, Conservative was the most sucesssfull party in UK.  